<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/BINANCE_SPOT_HFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ccxt

import ccxt
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import random

# Binance API credentials
api_key = 'your_api_key'
api_secret = 'your_api_secret'

# Initialize Binance client
exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
})

# Parameters for mid price simulation:
S0 = 100. # initial price
T = 1.0  # time in days
sigma = 2 # volatility
M = 200 # number of time steps
dt = T/M # time step
gamma = 0.1 # risk aversion
k = 1.5
A = 140
I = 1

# Fetch historical BTC/USDT data
def fetch_historical_data():
    ohlcv = exchange.fetch_ohlcv('BTC/USDT', timeframe='1m', limit=M+1)  # Fetching minute-level data
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Fetch the data
data = fetch_historical_data()
S = data['close'].values
M = len(S) - 1
T = M / (24 * 60)  # Adjust the total time based on fetched data

# Results:
AverageSpread = []
Profit = []
Std = []

for i in range(1):

    ### reservation price:
    ### r(s,t) = s - q * gamma * sigma**2 * (T-t)

    S_sim = np.zeros((M+1,I))
    Bid = np.zeros((M+1,I))
    Ask = np.zeros((M+1,I))
    ReservPrice = np.zeros((M+1,I))
    spread = np.zeros((M+1,I))
    deltaB = np.zeros((M+1,I))
    deltaA = np.zeros((M+1,I))
    q = np.zeros((M+1,I))
    w = np.zeros((M+1,I))
    equity = np.zeros((M+1,I))
    reserve_relation = np.zeros((M+1,I))

    S_sim[0] = S[0]
    ReservPrice[0] = S[0]
    Bid[0] = S[0]
    Ask[0] = S[0]
    spread[0] = 0
    deltaB[0] = 0
    deltaA[0] = 0
    q[0] = 0 # position
    w[0] = 0 # wealth
    equity[0] = 0

    for t in range(1, M+1):
        S_sim[t] = S[t]
        ReservPrice[t] = S[t] - q[t-1] * gamma * (sigma ** 2) * (T - t/float(M))
        spread[t] = gamma * (sigma **2) * (T - t/float(M)) + (2/gamma) * math.log(1 + (gamma/k))
        Bid[t] = ReservPrice[t] - spread[t]/2.
        Ask[t] = ReservPrice[t] + spread[t]/2.

        deltaB[t] = S[t] - Bid[t]
        deltaA[t] = Ask[t] - S[t]

        lambdaA = A * np.exp(-k * deltaA[t])
        ProbA = lambdaA * dt
        fa = random.random()

        lambdaB = A * np.exp(-k * deltaB[t])
        ProbB = lambdaB * dt
        fb = random.random()

        if ProbB > fb and ProbA < fa:
            q[t] = q[t-1] + 1
            w[t] = w[t-1] - Bid[t]
            # Account for trading fees
            w[t] -= Bid[t] * 0.001  # Assuming 0.1% fee

        if ProbB < fb and ProbA > fa:
            q[t] = q[t-1] - 1
            w[t] = w[t-1] + Ask[t]
            # Account for trading fees
            w[t] -= Ask[t] * 0.001  # Assuming 0.1% fee

        if ProbB < fb and ProbA < fa:
            q[t] = q[t-1]
            w[t] = w[t-1]

        if ProbB > fb and ProbA > fa:
            q[t] = q[t-1]
            w[t] = w[t-1] - Bid[t]
            w[t] = w[t] + Ask[t]

        equity[t] = w[t] + q[t] * S[t]

    AverageSpread.append(spread.mean())
    Profit.append(equity[-1])
    Std.append(equity[-1])

print("                   Results              ")
print("----------------------------------------")

print("%14s %21s" % ('statistic', 'value'))
print(40 * "-")
print("%14s %20.5f" % ("Average spread :", np.array(AverageSpread).mean()))
print("%16s %20.5f" % ("Profit :", np.array(Profit).mean()))
print("%16s %20.5f" % ("Std(Profit) :", np.array(Std).std()))

# Plots:
x = np.linspace(0., T, num= (M+1))

fig = plt.figure(figsize=(10,8))
plt.subplot(2,1,1) # number of rows, number of columns, number of the subplot
plt.plot(x, S_sim[:], lw=1., label='S')
plt.plot(x, Ask[:], lw=1., label='Ask')
plt.plot(x, Bid[:], lw=1., label='Bid')
plt.grid(True)
plt.legend(loc=0)
plt.ylabel('P')
plt.title('Prices')
plt.subplot(2,1,2)
plt.plot(x, q[:], 'g', lw=1., label='q') # plot 2 lines
plt.grid(True)
plt.legend(loc=0)
plt.axis('tight')
plt.xlabel('Time')
plt.ylabel('Position')

# Histogram of profit:
plt.figure(figsize=(7,5))
plt.hist(np.array(Profit), label=['Inventory strategy'], bins=100)
plt.legend(loc=0)
plt.grid(True)
plt.xlabel('pnl')
plt.ylabel('number of values')
plt.title('Histogram')

# PNL:
plt.figure(figsize=(7,5))
plt.plot(np.array(equity), label='Inventory strategy')
plt.legend(loc=0)
plt.grid(True)
plt.xlabel('pnl')
plt.ylabel('number of values')
plt.title('Profit')


ExchangeNotAvailable: binance GET https://api.binance.com/sapi/v1/capital/config/getall?timestamp=1719395714168&recvWindow=10000&signature=118a6e11f859c32ff3b802b9cf56be08426eae90caf47e13675cad9858e68114 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}